In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('nlp').getOrCreate()
df = spark.read.csv('SMSSpamCollection', inferSchema = True, sep = '\t')
df.printSchema()

22/11/29 18:46:24 WARN Utils: Your hostname, codeStation resolves to a loopback address: 127.0.1.1; using 172.17.0.1 instead (on interface docker0)
22/11/29 18:46:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/29 18:46:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/11/29 18:46:29 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


AnalysisException: Path does not exist: file:/run/media/solverbot/repoA/gitFolders/moreDE/PySpark-and-MLlib/SMSSpamCollection

In [5]:
df = df.withColumnRenamed('_c0', 'class').withColumnRenamed('_c1', 'text')
df.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



In [6]:
from pyspark.sql.functions import length

df = df.withColumn('length', length(df['text']))
df.show(3)

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
+-----+--------------------+------+
only showing top 3 rows



In [7]:
df.groupBy('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



In [8]:
from pyspark.ml.feature import (CountVectorizer, Tokenizer, 
                                StopWordsRemover, IDF, StringIndexer)

In [10]:
tokenizer = Tokenizer(inputCol = 'text', outputCol = 'token_text')
stop_remove = StopWordsRemover(inputCol = 'token_text', outputCol = 'stop_token')
count_vec = CountVectorizer(inputCol = 'stop_token', outputCol = 'c_vec')
idf = IDF(inputCol = 'c_vec', outputCol = 'tf_idf')
ham_spam_to_numeric = StringIndexer(inputCol = 'class', outputCol = 'label')

In [13]:
from pyspark.ml.feature import VectorAssembler

clean_up = VectorAssembler(inputCols = ['tf_idf', 'length'], outputCol = 'features')

In [7]:
import pyspark
from pyspark.ml.classification import NaiveBayes

nb = NaiveBayes(featuresCol='new','old')

SyntaxError: positional argument follows keyword argument (4224635255.py, line 4)

In [16]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[ham_spam_to_numeric, tokenizer, stop_remove, count_vec, idf, clean_up])

In [18]:
cleaner = pipeline.fit(df)
clean_df = cleaner.transform(df)
clean_df = clean_df.select('label', 'features')
clean_df.show(3)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,297,...|
|  1.0|(13424,[2,13,19,3...|
+-----+--------------------+
only showing top 3 rows



In [23]:
train, test = clean_df.randomSplit([0.7, 0.3])

In [24]:
df.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)



In [29]:
spam_detector = nb.fit(train)
predictions = spam_detector.transform(test)
predictions.show(3)

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,2,41,...|[-1061.8164592374...|[1.0,1.5335777452...|       0.0|
|  0.0|(13424,[0,1,5,20,...|[-815.00170676097...|[1.0,3.5993277744...|       0.0|
|  0.0|(13424,[0,1,9,14,...|[-539.27732500216...|[1.0,6.4723637628...|       0.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 3 rows



In [31]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator()
print("Test Accuracy: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "accuracy"})))

Test Accuracy: 0.9149191132414619
